In [ ]:
include("CRNExplore.jl") # NOTE never ever use t as a variable anywhere in this notebook!

In [ ]:
N = 3
tspan = (0., 20.)
np = count_parameters(N)
p0 = rand(np)
crn_p = assemble_opt_parameters_and_varables(p0, N)
crn = create_reactions(N)
prob = ODEProblem(crn, crn_p.u0, tspan, crn_p.p, maxiter=100)


input = 1.
intensity = 20.
perturb! = make_perturbation_event(input, intensity)

target = N
sensitivity_dt = 1.
sensitivity_offset = 0.4
steady_d = 1.0
K = 10

l1 = 0.0#10.0
l2 = 0.0
l3 = 1.0
l4 = 0.

rtol = 1e-12
atol = 1e-12
maxiters = 2000

absolute = true
use_adagrad = true

s_loss = stochastic_loss(K, prob, perturb!, tspan, target, tspan[2]/2., tspan[2], sensitivity_dt, sensitivity_offset, steady_d, l1, l2, l3, l4, absolute, rtol, atol, maxiters)

ALPHA = 0.1
NITER = 10
kill_on_clip = true
optsol = gradient_descent(ALPHA, NITER, p0, s_loss, true, true, 0.1/ALPHA, use_adagrad, kill_on_clip)
print("done")

In [ ]:
#continue from optsol
optsol = gradient_descent(ALPHA, NITER, optsol.minimizer, s_loss, true, true, 0.1/ALPHA, use_adagrad, kill_on_clip)
print("done")

### Visualization

In [ ]:
using Loess

In [ ]:
typeof(optsol.losses)

In [ ]:
plot(1:length(optsol.losses), optsol.losses, label="loss", xlabel="iteration", ylabel="loss", title="Loss vs iteration", lw=2, legend=:topright)
# smooth interpolation of the loss
model = loess(1:length(optsol.losses), optsol.losses, span=0.5)
predictions = predict(model, 1:length(optsol.losses))
plot!(1:length(optsol.losses), predictions, label="smoothed loss", lw=2, line=:dash)

In [ ]:
plot(1:length(optsol.grads), vec2mat(optsol.grads)[:,1:end], label="gradients", xlabel="iteration", ylabel="gradients", title="gradients vs iteration", lw=2, legend=false)

In [ ]:
plot(1:length(optsol.pars), vec2mat(optsol.pars)[:,1:end], label="parameters", xlabel="iteration", ylabel="parameters", title="parameters vs iteration", lw=2, legend=false)

In [ ]:
histogram(optsol.pars[argmin(optsol.losses)], label="parameters", xlabel="parameters", ylabel="frequency", title="parameters distribution", lw=2, legend=false, bins=100)

In [ ]:
length(findall(x -> x .> 0., optsol.pars[argmin(optsol.losses)]))

In [ ]:
best_pars = length(optsol.pars) 
#best_pars = argmin(optsol.losses)
println("ok")

In [ ]:
# test Dynamics
p = optsol.pars[best_pars]
#p[:30] = 100.
perturbation = 2.
input = 1.
prob_p = assemble_opt_parameters_and_varables(p, N)
max_t = 20.
n_perturb = 2
condition = [ max_t - max_t/n_perturb*(i) for i in 1:n_perturb-1]
affect! = make_perturbation_event(input, perturbation)
prob = ODEProblem(crn, prob_p.u0, (0., max_t), prob_p.p)
ps_cb = PresetTimeCallback(condition, affect!)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8, callback=ps_cb)
loss = s_loss(p)
plt1 = plot(sol.t[1:end], vec2mat(sol.u[1:end])[:,1:end]; lw = 1)
plt2 = plot(sol.t[1:end], vec2mat(sol.u[1:end])[:,N]; lw = 1)
#plot(plt1, plt2, layout=(2,1), legend=false, title = "L:$(loss), P$(perturbation)")
plot(plt1)

In [ ]:
r = vec2mat(sol.u)[end,N]
plot(sol.t, vec2mat(sol.u)[:,N]; lw = 1, ylims=(r-0.2, r+0.2), label="y3", color="green")

### Symbolic things

In [ ]:
# helper functions for substituting symbols in dictionaries
unsym = (x) -> eval(Meta.parse(string(x)))
unsym_dict = (d) -> Dict([unsym(k) => v for (k,v) in d])
# compute the symbolic jacobian
ode_sys = convert(ODESystem, crn)

In [ ]:
par_for_sym = assemble_opt_parameters_and_varables(optsol.pars[best_pars], N)
par_for_sym.p[:U] = sol.u[end][end]
par_u0 = unsym_dict([ ((par_for_sym.u0[i][1]), sol.u[end][i]) for i in 1:N])

In [ ]:
reduced_ode = substitute(ode_sys, unsym_dict(par_for_sym.p))

In [ ]:
# eval in u0
[
    substitute(equations(reduced_ode)[1].rhs, par_u0),
    substitute(equations(reduced_ode)[2].rhs, par_u0),
    substitute(equations(reduced_ode)[3].rhs, par_u0)
]

In [ ]:
jac = calculate_jacobian(reduced_ode)

In [ ]:
equations(reduced_ode)[1].rhs

In [ ]:
Symbolics.derivative(equations(reduced_ode)[1].rhs, x_3)

In [ ]:
Symbolics.derivative(equations(ode_sys)[1].rhs, eval(Meta.parse(string([k for (i, (k, v)) in enumerate(par_for_sym.p)][1]))))

In [ ]:
ode_sys

### Sensitivities and Symbolic Differentiation

In [ ]:
n_losses = 4

In [ ]:
include("SymbolicOps.jl")

In [ ]:
ode_sys

In [ ]:
Symbolics.derivative(equations(ode_sys)[3].rhs, k_1)

In [ ]:
crn = create_reactions(N)
ode_sys = convert(ODESystem, crn)
sensitivity(ode_sys, pars_l.p)

In [ ]:
t0 = 10.
t1 = 20.

In [ ]:
adaptation_loss_eval(adaptation_loss_symbolic(1), sol, 1, t0, t1)
adaptation_loss_symbolic(1)

In [ ]:
sensitivity_loss_eval(sensitivity_loss_symbolic(1), sol, 1, 1., 2., 0.2)
sensitivity_loss_symbolic(1)

In [ ]:
steady_state_loss_eval(steady_state_loss_symbolic(1), sol, 1., 1., 0.5)#
#steady_state_loss_symbolic(N)

In [ ]:
L1_loss_eval(L1_loss_symbolic(3,par_for_sym.p), par_for_sym.p)

In [ ]:
N=3
t0 = 10.
t1 = 20.
losses_arguments = [
    (
        eval = sensitivity_loss_eval,
        sym = sensitivity_loss_symbolic(N),
        args = (sol, 1, 1., 2., 0.2), # sol, t0, t1, p, d
        weight = 1.
    ),
    (
        eval = steady_state_loss_eval,
        sym = steady_state_loss_symbolic(N),
        args = (sol, 1., 2., 0.5), # sol, t0, t1, f_ss
        weight = 2.
    ),
    (
        eval = L1_loss_eval,
        sym = L1_loss_symbolic(N, par_for_sym.p), # N, p
        args = ([par_for_sym.p]), # p
        weight = 0.01
    ),
    (
        eval = adaptation_loss_eval,
        sym = adaptation_loss_symbolic(N), # N
        args = (sol, 1, t0, t1), # sol, target, t0, t1
        weight = 1.
    )
]

total_loss_sym = total_loss_symbolic(losses_arguments)

In [ ]:
total_loss_eval(losses_arguments)

In [ ]:
ode_outs = [
    [o_t0, o_t1, o_t0pdt, o_t1];
    [at_t0[i] for i in 1:N];
    [at_t1[i] for i in 1:N];
    [at_t0_d[i] for i in 1:N];
    [at_t1_d[i] for i in 1:N]
]
[Symbolics.derivative(total_loss_sym, p) for p in ode_outs]

In [ ]:
S = sensitivity(ode_sys, par_for_sym.p)

In [ ]:
plot(sol.t, vec2mat(sol.u)[:,1:end])

In [ ]:
n_losses=4
include("SymbolicOps.jl")
target = N
crn = create_reactions(N)
ode_crn = convert(ODESystem, crn)
np = count_parameters(N)
t0 = 10.
t1 = 20.
input = 1.
perturbation = 1.
pars_v = [rand() for i in 1:np]
pars_l = assemble_opt_parameters_and_varables(pars_v, N)
# function prepare_args(sol, target, t0, t1, pars_l)
#     return [
#         (
#             eval = sensitivity_loss_eval,
#             sym = sensitivity_loss_symbolic(1),
#             args = (sol, target, t0, 0.2, 0.5), # sol, target, t0, p, d
#             weight = 1.
#         ),
#         (
#             eval = steady_state_loss_eval,
#             sym = steady_state_loss_symbolic(1),
#             args = (sol, t0, t1, 0.5), # sol, t0, t1, f_ss
#             weight = 1.
#         ),
#         (
#             eval = L1_loss_eval,
#             sym = L1_loss_symbolic(N, pars_l.p), # N, p
#             args = ([pars_l.p]), # p
#             weight = 0.01
#         ),
#         (
#             eval = adaptation_loss_eval,
#             sym = adaptation_loss_symbolic(1), # N
#             args = (sol, target, t0, t1), # sol, target, t0, t1
#             weight = 1.
#         )
#     ]
# end

# function evaluate_the_loss(sol, target, t0, t1, parameters)
#     losses_arguments = prepare_args(sol, target, t0, t1, parameters)
#     total_loss_sym = total_loss_symbolic(losses_arguments)
#     return total_loss_eval(losses_arguments)
# end

sol = run(crn, pars_v, pars_l, 1, 1., t0, t1)
evaluate_the_loss(sol, target, t0, t1, pars_l)
S = sensitivity(ode_crn, pars_l.p)


In [ ]:
jacobian_pars(ode_crn, sol, N, t0, t1, pars_l.p, 0.5, 0.5)

In [ ]:
size(S)

In [ ]:

extended_ode = make_sensitivity_ode(ode_crn, par_for_sym.p)

# function label(sol, var_to_name)
#     m = vec2mat(sol.u)
#     u = [] 
#     i=1
#     for (k,v) in var_to_name
#         if length(string(k))<2 || !(string(k)[1:2] == "k_")
#             push!(u, k => m[:,i])
#             i+=1
#         end
#     end
#     return (
#         t = sol.t,
#         u = Dict(u)
#     )
# end

# function label_with(var_to_name)
#     function l(sol_u)
#         u = [] 
#         i=1
#         for (k,v) in var_to_name
#             if length(string(k))<2 || !(string(k)[1:2] == "k_")
#                 push!(u, k => sol_u[i])
#                 i+=1
#             end
#         end
#         return Dict(u)
#     end
#     return l
# end

# old
# function sensitivity_from_ode(ode, sol, t)
#     return vec2mat([sol(t, idxs=[ode.var_to_name[Symbol("ks_$(i)_$(j)")] for j in 1:count_parameters(N)]) for i in 1:N])
# end

#structural_simplify(extended_ode)
ret = run_extended(extended_ode, pars_v, pars_l, 1., 1., t0, t1)
#plot(ret.t, vec2mat(ret.u)[:,1:N+1])
#plot(ret.t, ret.u[:x_1],legend=false)
#extend_u0(extended_ode, pars_l.u0, pars_l.p, N).p
#print(extended_ode)
state_from_ode(extended_ode, ret, 1.)
control_from_ode(extended_ode, ret, 1.)
sensitivity_from_ode(extended_ode, ret, 1.)

In [ ]:
extended_ode.systems

In [ ]:
crn

In [ ]:
sensitivity(ode_crn, pars_l.p)

In [ ]:
sum(sensitivity(ode_crn, pars_l.p), dims=1)

In [ ]:
for i in 1:100
    sensitivity_from_ode(extended_ode, ret, 0.01)
end

In [ ]:
# for i in 1:100
#     sensitivity_from_ode(extended_ode, ret, 0.01)
# end
sensitivity_from_ode(extended_ode, ret, 1.)

In [ ]:
# function dict_indexes(ode)
#     out = []
#     for k in keys(ode.var_to_name)
#         if length(string(k)) > 1 && string(k)[1:2] == "ks"
#             splitted_str = split(string(k), "_")
#             push!(out, (parse(Int, splitted_str[2]), parse(Int, splitted_str[3])))
#         end
#     end
#     out
# end



In [ ]:
# function symbolic_loss(sol, target, t0, t1, parameters)
#     losses_arguments = prepare_args(sol, target, t0, t1, parameters)
#     total_loss_sym = total_loss_symbolic(losses_arguments)
#     return total_loss_sym
# end

# s_loss = symbolic_loss(ret, 1, t0, t1, pars_l)

In [ ]:
loss_derivatives[o_t0]

In [ ]:
steady_state_loss_symbolic(N)

In [ ]:
import Pkg; Pkg.add("SimpleDiffEq")

### Testing the whole thing

In [ ]:
include("CRNExplore.jl")
include("SymbolicOps.jl")
target = N
crn = create_reactions(N)
ode_crn = convert(ODESystem, crn)
np = count_parameters(N)
t0 = 10.
t1 = 20.
input = 1.
perturbation = 1.
pars_v = [rand() for i in 1:np]
pars_l = assemble_opt_parameters_and_varables(pars_v, N)
weights = [1., 0., 0., 0.] #[10., 0.1, 0.01, 100.] this descends smoothly
p=0.3
d=1.
f_ss=0.5
args = prepare_args(nothing, target, t0, t1, pars_l, weights, p, d, f_ss)
ext_ode = make_sensitivity_ode(ode_crn, pars_l.p)

s_loss = total_loss_symbolic(args)

loss_derivatives = Dict([
        [
            o_t0 => (Symbolics.derivative(s_loss, o_t0)),
            o_t1 => (Symbolics.derivative(s_loss, o_t1)),
            o_t0pdt => (Symbolics.derivative(s_loss, o_t0pdt))
        ]
        [at_t0[i] => Symbolics.derivative(s_loss, at_t0[i]) for i in 1:length(at_t0)];
        [at_t1[i] => Symbolics.derivative(s_loss, at_t1[i]) for i in 1:length(at_t1)];
        [at_t0_d[i] => Symbolics.derivative(s_loss, at_t0_d[i]) for i in 1:length(at_t0_d)];
        [at_t1_d[i] => Symbolics.derivative(s_loss, at_t1_d[i]) for i in 1:length(at_t1_d)];
])

In [ ]:
s_loss

In [ ]:
alpha = 0.1
loss_args = args
losses = []
K = 5
par_recoder = []
grad_recoder = []
#push!(losses, total_loss_eval(loss_args))
push!(par_recoder, pars_v)
push!(grad_recoder, zeros(np))
grad_history = vec(zeros(length(pars_v)))
grad_valid = vec(ones(length(pars_v)))
for i in 1:30
    grad = zeros(np)
    push!(losses, 0.)
    for _ in 1:K
        sol = run_extended(ext_ode, pars_v, pars_l, input, perturbation, t0, t1)
        loss_args = update_args(sol, 1, t0, t1, pars_l, loss_args, p, d, f_ss)
        losses[end] += total_loss_eval(loss_args)
        jacobian = jacobian_pars(ext_ode, loss_args, loss_derivatives, sol, N, t0, t1, pars_v, f_ss, d, [[Symbol("x_$(i)") for i in 1:N]..., Symbol("U")])
        grad += vec([v.val for v in jacobian.sensitivity])
    end
    losses[end] /= K
    grad /= K
    grad = grad .* grad_valid
    #grad = sign.(grad).*min.(abs.(grad), 1.)
    #grad /= maximum(abs.(grad))
    # upate params 
    #pars_v = max.(0., pars_v .- (alpha .* grad))
    delta_g = adagrad_update_get_coefficient(pars_v, grad, grad_history, alpha)
    pars_v = max.(0., pars_v - (delta_g .* grad)).*grad_valid
    grad_valid = pars_v .> 0.
    pars_l = assemble_opt_parameters_and_varables(pars_v, N)
    # sol, target, t0, t1, pars_l, old_args, p, d, f_ss
    loss_args = update_args(nothing, 1, t0, t1, pars_l, loss_args, p, d, f_ss)
    push!(par_recoder, pars_v)
    push!(grad_recoder, grad)
end

In [ ]:
loss_derivatives[o_t0]

In [ ]:
[0,0,1]'*sensitivity_from_ode(ode_sys, test_sol_full, t0) +
[0,0,1]'*sensitivity_from_ode(ode_sys, test_sol_full, t1)

In [ ]:
test_sol = run(crn, pars_v, pars_l, input, perturbation, t0, t1)
test_sol_full = run_extended(ext_ode, pars_v, pars_l, input, perturbation, t0, t1)
out = jacobian_pars(ext_ode, loss_args, loss_derivatives, test_sol, test_sol_full, N, t0, t1, pars_v, f_ss, d, [[Symbol("x_$(i)") for i in 1:N]..., Symbol("U")])
out.fwd_pass

Symbolics.substitute(loss_derivatives[o_t0], out.fwd_pass)
out.sensitivity

In [ ]:
plot(1:length(grad_recoder), vec2mat(grad_recoder), legend=false, title="Gradient", xlabel="Iteration", ylabel="Gradient")

In [ ]:
plot(1:length(par_recoder), vec2mat(par_recoder), legend=false, title="Parameters", xlabel="Iteration", ylabel="Parameter value")

In [ ]:
plot(1:length(losses), [losses[i].val for i in 1:length(losses)])

In [ ]:
sol = run_extended(ext_ode, pars_v, pars_l, input, perturbation, t0, t1)
plot(sol.t, vec2mat(sol.u)[:,1:4])

In [ ]:
plot(sol.t, vec2mat(sol.u)[:,3], legend=false, title="Output", xlabel="Time", ylabel="Concentration", xrange=(0, 20))

In [ ]:
extended_ode.var_to_name[:ks_1_1]

In [ ]:
ModelingToolkit.get_iv(extended_ode)

In [ ]:
# simple gd loop

for i in 1:100
    sol = run(crn, pars_v, pars_l, 1, 1., t0, t1)
    loss = evaluate_the_loss(sol, target, t0, t1, pars_l)
    grads = [Symbolics.derivative(loss, p) for p in ode_outs]
    pars_v = [pars_v[i] - 0.01 * grads[i] for i in 1:np]
    pars_l = assemble_opt_parameters_and_varables(pars_v, N)
end

In [ ]:
simulate(

In [ ]:
pars = assemble_opt_parameters_and_varables(optsol.pars[best_pars], N)

In [ ]:
jacobian_pars(sol, t0, t1, par_for_sym.p)

### NFB vs IFF

In [ ]:
using Symbolics
using LinearAlgebra
function joint_jacobian(i, j, jac, initial_conditions)
    A_ij = substitute(jac[i, j], unsym_dict(initial_conditions))
    return A_ij
end
A_21 = joint_jacobian(2, 1, jac, par_for_sym.u0)
A_32 = joint_jacobian(3, 2, jac, par_for_sym.u0)
A_22 = joint_jacobian(2, 2, jac, par_for_sym.u0)
A_31 = joint_jacobian(3, 1, jac, par_for_sym.u0)
A_22*A_31 - A_21*A_32

In [ ]:
det(A_22*A_31 - A_21*A_32)

In [ ]:
A_22*A_31

In [ ]:
A_21*A_32

### More general sanity check

In [ ]:
# make a Matrix
DuA = [1. 0. 0.;]'
h = [0. 0. 1.;]

In [ ]:
phi = -h* det(jac) * inv(jac) * A
substitute(phi, unsym_dict(par_for_sym.u0))

In [ ]:
det

In [ ]:
eval(x)

### Example: integration only 

In [ ]:
N = 3
tspan = (0., 20.)
np = count_parameters(N)
p0 = rand(np)
crn_p = assemble_opt_parameters_and_varables(p0, N)
crn = create_reactions(N)
prob = ODEProblem(crn, crn_p.u0, tspan, crn_p.p)
input = 1.
intensity = 1.
perturb! = make_perturbation_event(input, intensity)
reltol = 1e-5
absto = 1e-5
maxiters = 1000
sol = integrate(prob, tspan, perturb!, reltol, absto, maxiters)
plot(sol.t, sol.u[1,:])
plot(sol.t, vec2mat(sol.u)[:,1:N], legend=false, title="Dynamics (npt=$(length(sol.t)))")

In [ ]:
integrate(prob, tspan, perturb!, reltol, absto, maxit)

### Evolution stuff

In [ ]:
include("CRNevo.jl")
genetic_pool_size = 100
elite = 20
worst = 40
death_rate = 0.1
mutation_rate = 0.55
gradient_mutation_rate = 0.05
duplication_rate = 0.1
crossover_rate = 0.2
max_generations = 100
loss_rep = 100
p_cross = 0.5

parameter_pool = [assemble_opt_parameters_and_varables([rand() for _ in 1:np], N) for _ in 1:genetic_pool_size]
is_updated = [false for _ in 1:genetic_pool_size]
fitness = [0. for _ in 1:genetic_pool_size]

ALPHA = 0.1
NITER = 30
use_adagrad = true
K = 20

s_loss = stochastic_loss(K, prob, perturb!, tspan, target, tspan[2]/2., tspan[2], sensitivity_dt, sensitivity_offset, steady_d, l1, l2, l3, l4, absolute, rtol, atol, maxiters)
mutate_with_GD = (p) -> gradient_descent(ALPHA, NITER, p, s_loss, true, false, 0.01/ALPHA, use_adagrad).minimizer

dp = 0.01

state = (pool = parameter_pool, is_updated = is_updated, fitness = fitness, history = (best_loss = [], mean_loss = []))
print("done")
prob = ODEProblem(crn, parameter_pool[1].u0, tspan, parameter_pool[1].p)

In [ ]:
using ProgressBars
max_generations = 100
for i in ProgressBar(1:max_generations)
    state = evolve(crn, prob, s_loss, perturb!, state, tspan, dp, genetic_pool_size, elite, worst, death_rate, mutation_rate, gradient_mutation_rate, mutate_with_GD, duplication_rate, crossover_rate, p_cross, loss_rep)
end

In [ ]:
plot(state.history.best_loss, label="best loss", xlabel="generation", ylabel="loss", title="Best loss vs generation", lw=2, legend=:bottomright)
plot!(state.history.mean_loss, label="mean loss", lw=2, line=:dash)

In [ ]:
using ModelingToolkit

In [ ]:
ode_sys = convert(ODESystem, crn)

In [ ]:
jac = calculate_jacobian(ode_sys)

In [ ]:
[x[1] for x in crn_p.u0]

In [ ]:
[Meta.parse(string(x[1])*"(t)") for x in crn_p.u0] 

In [ ]:
merge(crn_p.p, Dict([Meta.parse(string(x[1])*"(t)") for x in crn_p.u0] .=> [x[2] for x in crn_p.u0]), Dict(["U" => 1.0]) )

In [ ]:
@variables testv_1 testv_2
substitute(testv_1 + testv_2, Dict([eval(Meta.parse("testv_1")) => 1.0, testv_2 => 2.0]))

In [ ]:
substitute(k_1, Dict(["k_1" => 1.0]))

In [ ]:
[Meta.parse(string(x[1])*"(t)") for x in crn_p.u0]

In [ ]:
eval(Meta.parse("testv_1"))

In [ ]:
[eval(Meta.parse(string(x[1]))) for x in crn_p.u0]

In [ ]:
unsym = (x) -> eval(Meta.parse(string(x)))
unsym_dict = (d) -> Dict([unsym(k) => v for (k,v) in d])

In [ ]:
substitute(jac, merge(unsym_dict(crn_p.p), unsym_dict(crn_p.u0)))

In [ ]:
Symbolics.derivative(jac, [:x_1], p)

In [ ]:
create_reactions(N)